# Kaggle-Tutorial

## Intro to Machine Learning

---

### 과소적합Underfitting 과 과대적합Overfitting

더 나은 성능을 위해 모델을 미세조정하자.

원본: [Underfitting and Overfitting](https://www.kaggle.com/dansbecker/underfitting-and-overfitting)

---
이 단계를 마치면 과소적합Underfitting 과 과대적합Overfitting의 개념을 이해하고 이러한 구상을 적용하여 모델을 보다 정확하게 만 수 있다.

### 다른 모델을 사용한 실험

이제 신회할 수 있는 모델 정확도 측정 방법이 생겼으으모 다른 모델로 실험하고 어떤 모델이 가장 좋은 예측을 할 수 있는지 볼 수 있다.
하지만, 당신이 사용할 수 있는 다른 모델은 무엇있을까?


Scikit-Learn의 [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)에서
당신은 의사결정트리의 다양한 옵션(당신이 원하고 필료한 시간 보다 더 많은)이 있음을 볼 수 있다. 가장 중요한 옵션은 트리의 깊이를 결정하는 것이다.
[이 코스의 첫번째 단원](https://www.kaggle.com/dansbecker/how-models-work)에서 나무의 깊이는 예측에 도달하기 위해 얼마나 많이 분할 할 것인가를 측정하는 척도임을 기억하라.
이건 비교적 앝은 수준의 트리이다.

![](http://i.imgur.com/R3ywQsR.png)

실제로, 한 트리가 top level(모든 집들)과 leaf 사이에 10개의 분할이 있는 것 드문 일이 아니다.
트리가 깊어질수록 데이터셋은 적은 집들을 가리키는 잎들leaves로 부터 잘려져간다.
만약 트리가 1개의 분할만을 가진다면, 데이터들은 2그룹으로 나뉜다. 만약 각 그룹이 다시 갈라지다면, 우리는 4개의 그룹을 갖게 될 것이다.
각각의 그룹을 다시 분할하면 8개의 그룹이 만들어진다. 각 단계별로 분할을 추가하여 그룹의 수를 2배로 늘리되면 10번째 층에서는
$$2^{10}$$개의 그룹을 가지게 된다. 이는 1024개의 잎들이다.

우리가 집들을 여러개의 잎들로 나눌 때, 각 잎에는 점점 더 적은 수의 집들이 배치될 것이다. 매우 적은 수들의 집들이 할당된 잎들은
그 집의 실제값에 상당히 가까운 예측을 하지만, 새로운 데이터에 대해서는 신뢰핟기 어려운 예측을 할 수 있다(각 예측의 경우들은 오직 아주 적은 수의 집들만을 근거하기 때문).

이엇이 과적합Overfitting이라 불리는 현상인데, 모델이 훈련데이터에는 거의 환벽하게 일치하지만, 검증validation 및 다른 새로운 데이터에서는 성능이 떨어진다.
반대로 우리가 트리를 너무 낮게 만든다면, 그것은 집들을 분명한 그룹으로 나눌 수 없게 된다.

극단적으로, 만약 한 트리가 오지 2개나 4개의 그룹으로만 나눈다면, 각 그룹은 여전히 넓은 범주의 집들을 가지데 될 것이다.
예측 결과는 비록 학습데이터셋이라 할지라도(물론, 같은 이유로 검증데이터에서도 마찬가지로 좋지 않을 것이다) 대부분의 집들은 그 예측이 크게 벗어날 수 있다.
모델이 데이터에서 중요한 구별이나 패턴을 찾아내지 못해, 교육욕 데이터셋에서도 성능이 형편없게 됨을 과소적합Underfitting이라 부른다.

검증데이터들을 통해 추정하는 새로운 데이터에 대한 정확성에 대해 다루기 위해서, 우리는 과소적합Underfitting 과 과대적합Overfitting 사이의 최적의 지점을 찾고자 한다.
시각적으로, 우리는 아래 그림의 (빨간색) 검증Validation 곡선에서의 저점low point을 원한다.

![](http://i.imgur.com/AXSEOfI.png)


### 예시
트리의 깊이를 조절할는 몇가지 대안이 있으며, 이러한 방식들은 다른 방식들에 비해 더 깊은 트리를 가질 수 있도록 많이들 허용한다.
그러나 *max_leaf_nodes* 인수argument는 메우 실용적인 방식으로 과소적합Underfitting 과 과대적합Overfitting 문제를 다룰 수 있는 방법을 제공한다.
위의 그래프에서 볼 때, 모델이 더 많은 잎을 만들수록 과소적합영역에서 과대 적합 영역으로 이동한다.

우리는 MAE점수와 *max_leaf_nodes*의 차이를 비교하는 것을 돕기위해 유틸리티 함수를 사용할 수 있다.


In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

데이터는 사용자가 이미 보고 작성한 코드를 사용하여 train_X, val_X, train_y 및 val_y에 로드된다.

In [2]:
# Data Loading Code Runs At This Point
import pandas as pd

# Load data
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)
# Filter rows with missing values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea',
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

ModuleNotFoundError: No module named 'pandas'

우리는 for-loop를 이용하여 *max_leaf_nodes*을 통해 만들어진 다른 모델의 정확도를 비교할 수 있다.

In [ ]:
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5            Mean Absolute Error:  347380
Max leaf nodes: 50  		 Mean Absolute Error:  258171
Max leaf nodes: 500  		 Mean Absolute Error:  243495
Max leaf nodes: 5000  		 Mean Absolute Error:  254983

나열된 옵션들 중 500개가 최적의 잎 개수임을 볼 수 있다.

### 결론
이것만큼은 가져갈 것: 모델은 다음 중 하나로 어려움 겪을 수 있다.

과적합: 미래에 재발하지 않을 가짜 패턴을 포착하여 예측 정확도가 떨어지거나
과소 적합: 관련 패턴을 캡처하지 못해 예측 정확도가 떨어짐

우리는 모델 교육에 사용되지 않는 **검증 데이터**를 사용하여 후보 모델condidate model의 정확도를 측정한다.
이것은 우리가 많은 후보 모델을 시도하고 가장 좋은 모델을 유지할 수 있게 해준다.

### 당신의 차례
[이전에 구축한 모델을 최적화]() 해보시오